In [18]:
import os
import pyspark
import pandas as pd
from functools import reduce
from pyspark.sql import SparkSession
from pyspark.sql.functions import max as _max, min as _min,round
from pyspark.sql.functions import *
from pyspark.sql.functions import col, sqrt,sum,avg,count,explode
from operator import add
from pyspark.sql.types import IntegerType
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.sql.functions import trim
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql.functions import *

In [16]:
spark.stop()

In [21]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '20g'), 
                                   ('spark.executor.instances', '12'),
                                   ('spark.driver.cores','1'), 
                                   ('spark.executor.cores', '4'), 
                                   ('spark.sql.broadcastTimeout', '36000'),
                                   ('spark.yarn.queue', 'root.araadh_q1.uhc_oa_pi_dev_sq1'),
                                   ('spark.driver.memory','5g')])
spark = SparkSession.builder.appName("Model-C").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
spark

# lead_gen table

In [3]:
lead_gen = spark.read.parquet("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/Final_HNP")

In [4]:
lead_gen.printSchema()

root
 |-- entity_type: integer (nullable = true)
 |-- DOS_P12M_PatCnt: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- HNP_RB_Reference_Date: string (nullable = true)
 |-- Platform_P12M_ClmPer: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CPT_P12M_Cnt: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CPT_P9M_Cnt: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CPT_P6M_Cnt: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CPT_P3M_Cnt: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Avg_P3M_PatCnt: double (nullable = true)
 |-- Avg_P6M_PatCnt: double (nullable = true)
 |-- Avg_P9M_PatCnt: double (nullable = true)
 |-- Avg_P12M_PatCnt: double (nullable = true)
 |-- Diag_P12M_Cnt: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Diag_P9M_Cnt: array (nullable = true)
 |    |-- element: strin

In [5]:
lead_gen.count()

1458

# De-duping

In [6]:
dedupe = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/de-dupe.csv",header=True)
dedupe.show()

+----------+----+
|  prov_npi|Team|
+----------+----+
|1649218173|PLRT|
|1427293281| AVS|
|1275791832|PLRT|
|1285660241| AVS|
+----------+----+



In [8]:
lead_gen.createOrReplaceTempView("lead_gen")

# Filter File

In [9]:
lead_release = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HBP_Lead_Release_new.csv",header=True)

In [10]:
lead_release.show(truncate=False)

+----+-----------------------------------------------------------------------+---+----+---+--------------------------------------+--------------------------------------------------------------------------------------+-----+--------------+------+
|Team|Features                                                               |DOS|DIAG|CPT|Numeric_Filters                       |Categorical_Filters                                                                   |Leads|Priority_Order|Dedupe|
+----+-----------------------------------------------------------------------+---+----+---+--------------------------------------+--------------------------------------------------------------------------------------+-----+--------------+------+
|PLRT|prov_npi,prov_name,entity_type,DOS_P12M_PatCnt,CPT_P9M_Cnt,Diag_P6M_Cnt|5  |3   |3  |Platform_P12M_ClmPer:[PNIC>80,CIRR>80]|Platform_P12M_ClmPer:[PNIC];Payer_ID_P12M_ClmPer:[95964];Payer_nm_P12M_ClmPer:[MEDICA]|10   |1             |All   |
|AVS |prov_npi,p

# Categorical Filters

In [11]:
lead_release = lead_release.withColumn("catfil_arr",split(lead_release["Categorical_Filters"],";"))
cat_filter = lead_release.select("catfil_arr").collect()[0][0]
cat_filter

['Platform_P12M_ClmPer:[PNIC]',
 'Payer_ID_P12M_ClmPer:[95964]',
 'Payer_nm_P12M_ClmPer:[MEDICA]']

In [18]:
#lr = lr.withColumn("cf_arr",split(lr["cat_fil"],","))
#a_cat2 = lr.select("cf_arr").collect()[0][0]
#a_cat2

In [12]:
lead_release = lead_release.withColumn("catfil_arr",split(lead_release["Categorical_Filters"],";"))
cat_filter = lead_release.select("catfil_arr").collect()[0][0]
cat_filter

col_name = ""
col_value = ""
if not (cat_filter is None):
    for i in cat_filter:
        col_name = i.split(":")[0].strip()
        col_value = i.split(":")[1].strip()
        col_value = col_value[1:len(col_value)-1]
        col_value_list = col_value.split(",")
        lead_gen = lead_gen.withColumn("cat_flag_"+col_name,concat_ws(",",lead_gen[col_name]))
        for j in col_value_list:
            print("j: ",j)
            lead_gen = lead_gen.withColumn("categorical_flag_"+j, lead_gen["cat_flag_"+col_name].rlike(j))
            lead_gen = lead_gen.withColumn("categorical_flag_"+j, lead_gen["categorical_flag_"+j].cast(StringType()))
else:
    print("Category Filter is Empty")

j:  PNIC
j:  95964
j:  MEDICA


In [13]:
lead_gen.select("Platform_P12M_ClmPer","categorical_flag_PNIC").show(10,False)

+--------------------------------------------------------------------------------+---------------------+
|Platform_P12M_ClmPer                                                            |categorical_flag_PNIC|
+--------------------------------------------------------------------------------+---------------------+
|[38.35:TOPS, 0.02:CIRR, 0.11:OPUL, 37.23:COSM, 24.25:FACS, 0.04:PNIC]           |true                 |
|[47.54:TOPS, 15.31:FACS, 1.67:PNIC, 0.08:OPUL, 0.03:FAOB, 35.36:COSM, 0.02:CIRR]|true                 |
|[37.3:TOPS, 0.04:FACS, 62.59:COSM, 0.04:FAOB, 0.01:OPUL, 0.02:PNIC]             |true                 |
|[0.92:OPUL, 28.84:COSM, 48.13:FACS, 3.31:CIRR, 0.04:FAOB, 18.74:TOPS]           |false                |
|[0.15:FACS, 0.04:OPUL, 77.78:TOPS, 0.03:CIRR, 22.0:COSM]                        |false                |
|[57.58:TOPS, 0.04:CIRR, 0.01:FAOB, 21.71:COSM, 20.55:FACS, 0.1:OPUL]            |false                |
|[89.04:FACS, 4.92:TOPS, 6.04:COSM]                    

# Numeric Filters

In [14]:
lead_release = lead_release.withColumn("num_fil_arr",split(lead_release["Numeric_Filters"],";"))
num_filter = lead_release.select("num_fil_arr").collect()[0][0]
num_filter

['Platform_P12M_ClmPer:[PNIC>80,CIRR>80]']

In [15]:
lead_release = lead_release.withColumn("num_fil_arr",split(lead_release["Numeric_Filters"],";"))
num_filter = lead_release.select("num_fil_arr").collect()[0][0]
num_filter

import re
col_name = ""
col_value = ""
if not(num_filter is None):
    for i in num_filter:
        i = i.strip()
        col_name = i.split(":")[0].strip()
        col_value = i.split(":")[1].strip()
        col_value = col_value[1:len(col_value)-1]
        col_value_list = col_value.split(",")
        col_str =col_name+"_str"
        print("col_name: ",col_name)
        print("col_value: ",col_value)
        print("col_value_list: ",col_value_list)
        lead_gen = lead_gen.withColumn(col_str,lead_gen[col_name].cast(StringType()))
        for j in col_value_list:
            j_list = re.split('([^a-zA-Z0-9])',j)
            col_name = j_list[0].strip()
            sign = j_list[1].strip()
            num = j_list[2].strip()
            #val = col_name[0:len(col_name)-1]
            #num = j.split(" ")[1].strip()
            #num = float(num)
            reg ="((\d+)|(\d+\.\d+))(?=:"+col_name+")" 
            print("j_list :",j_list)
            print("col_name: ",col_name)
            #print("cl:",cl)
            #print("val",val)
            #print("num",num)
            print("sign",sign)
            print("num",num)
            #a =cl+"_str"
            lead_gen = lead_gen.withColumn("num_"+col_name,regexp_extract(lead_gen[col_str],reg,1))
            lead_gen = lead_gen.withColumn("num_"+col_name,lead_gen["num_"+col_name].cast('float'))
            lead_gen = lead_gen.fillna(0)
            if sign==">":
                lead_gen = lead_gen.withColumn("numeric_flag_"+col_name, when(lead_gen["num_"+col_name] >= num,"true").otherwise("false"))
            else:
                lead_gen = lead_gen.withColumn("numeric_flag_"+col_name, when(lead_gen["num_"+col_name] <= num,"true").otherwise("false"))
else:
    print("Numeric Filter is empty")

col_name:  Platform_P12M_ClmPer
col_value:  PNIC>80,CIRR>80
col_value_list:  ['PNIC>80', 'CIRR>80']
j_list : ['PNIC', '>', '80']
col_name:  PNIC
sign >
num 80
j_list : ['CIRR', '>', '80']
col_name:  CIRR
sign >
num 80


In [45]:
lead_gen.select("prov_npi","Platform_P12M_ClmPer","numeric_flag_PNIC","numeric_flag_CIRR").show(truncate=False)

+----------+--------------------------------------------------------------------------------+-----------------+-----------------+
|prov_npi  |Platform_P12M_ClmPer                                                            |numeric_flag_PNIC|numeric_flag_CIRR|
+----------+--------------------------------------------------------------------------------+-----------------+-----------------+
|1275791832|[38.35:TOPS, 0.02:CIRR, 0.11:OPUL, 24.25:FACS, 37.23:COSM, 0.04:PNIC]           |false            |false            |
|1285660241|[47.54:TOPS, 15.31:FACS, 1.67:PNIC, 0.08:OPUL, 0.03:FAOB, 35.36:COSM, 0.02:CIRR]|false            |false            |
|1386665503|[37.3:TOPS, 0.04:FACS, 62.59:COSM, 0.04:FAOB, 0.01:OPUL, 0.02:PNIC]             |false            |false            |
|1174595466|[0.92:OPUL, 28.84:COSM, 48.13:FACS, 3.31:CIRR, 0.04:FAOB, 18.74:TOPS]           |false            |false            |
|1104265016|[0.15:FACS, 0.04:OPUL, 77.78:TOPS, 0.03:CIRR, 22.0:COSM]                      

In [16]:
lead_gen.createOrReplaceTempView("lead_gen")

# Extracting the filtered leads

In [17]:
lead_gen_columns = lead_gen.columns

fil_col_list = [x for x in lead_gen_columns if ("categorical_flag" in x or "numeric_flag" in x)]
print("fil_col_list:\n",fil_col_list)
for i in fil_col_list:
    lead_gen = lead_gen.withColumn(i,when(lead_gen[i]=="true","1").otherwise("0"))


fil_col_list:
 ['categorical_flag_PNIC', 'categorical_flag_95964', 'categorical_flag_MEDICA', 'numeric_flag_PNIC', 'numeric_flag_CIRR']


In [18]:
lead_gen.select('prov_npi','categorical_flag_PNIC', 'categorical_flag_95964', 'categorical_flag_MEDICA', 'numeric_flag_PNIC', 'numeric_flag_CIRR').show(20,False)

+----------+---------------------+----------------------+-----------------------+-----------------+-----------------+
|prov_npi  |categorical_flag_PNIC|categorical_flag_95964|categorical_flag_MEDICA|numeric_flag_PNIC|numeric_flag_CIRR|
+----------+---------------------+----------------------+-----------------------+-----------------+-----------------+
|1275791832|1                    |1                     |1                      |0                |0                |
|1285660241|1                    |0                     |1                      |0                |0                |
|1386665503|1                    |1                     |1                      |0                |0                |
|1174595466|0                    |0                     |1                      |0                |0                |
|1104265016|0                    |1                     |1                      |0                |0                |
|1053370742|0                    |1                     

In [19]:
lead_gen.createOrReplaceTempView("lead_gen")

In [20]:
temp_query=""
for i in fil_col_list:
    temp_query =temp_query+" and "+i+" ==0"
temp_query=temp_query[5:len(temp_query)]
print(temp_query)

lead_gen_fil = spark.sql("select a.* from lead_gen a where "+temp_query)

categorical_flag_PNIC ==0 and categorical_flag_95964 ==0 and categorical_flag_MEDICA ==0 and numeric_flag_PNIC ==0 and numeric_flag_CIRR ==0


In [68]:
lead_gen_fil.count()

63

# Fetching the number of dos,diag,cpt,leads etc. 

In [70]:
dos = int(lead_release.select("DOS").collect()[0][0])
diag = int(lead_release.select("DIAG").collect()[0][0])
cpt = int(lead_release.select("CPT").collect()[0][0])
leads= int(lead_release.select("Leads").collect()[0][0])
#tax= int(lr.select("TAX").collect()[0][0])
#pid= int(lr.select("PID").collect()[0][0])
#pnm =int(lr.select("PNM").collect()[0][0])
print(dos)
print(diag)
print(cpt)
print(leads)
#print(tax)
#print(pnm)
#print(pid)

5
3
3
10


# Sorting & Selecting no. of leads

In [124]:
lead_gen_fil =lead_gen_fil.orderBy(lead_gen_fil["pat_cnt_51"].desc(),lead_gen_fil["Avg_Max_PatCnt_Ratio"].desc()).limit(leads)
lead_gen_fil.createOrReplaceTempView("lead_gen_fil")

In [125]:
lead_gen_fil.select("prov_npi","pat_cnt_51","Avg_Max_PatCnt_Ratio").show(truncate=False)

+----------+----------+--------------------+
|prov_npi  |pat_cnt_51|Avg_Max_PatCnt_Ratio|
+----------+----------+--------------------+
|1255342184|4         |53.09               |
|1285044677|3         |45.38               |
|1164628129|2         |43.74               |
|1861693947|1         |57.92               |
|1417183542|1         |57.65               |
|1952481657|1         |55.68               |
|1528005196|1         |54.01               |
|1417301128|1         |50.5                |
|1447885702|1         |45.07               |
|1619413895|1         |43.42               |
+----------+----------+--------------------+



# Slicing  arrays

In [73]:
from pyspark.sql.functions import explode,col
array_columns = ['DOS_P12M_PatCnt','CPT_P12M_Cnt','CPT_P9M_Cnt','CPT_P6M_Cnt','CPT_P3M_Cnt','Diag_P12M_Cnt','Diag_P9M_Cnt','Diag_P6M_Cnt','Diag_P3M_Cnt']

for i in array_columns:
    print(i)
    temp = lead_gen.select("prov_npi",explode(i).alias("array_val"))
    temp = temp.withColumn("key",trim(split(temp["array_val"],":").getItem(0)))
    temp = temp.withColumn("key",temp["key"].cast("float"))
    window = Window.partitionBy(temp['prov_npi']).orderBy(temp["key"].desc())
    temp1 = temp.select('*', rank().over(window).alias('rank'))
    if "DOS" in i:
        temp1 = temp1.filter(temp1["rank"] <= dos)
    elif "CPT" in i:
        temp1 = temp1.filter(temp1["rank"] <= cpt)
    elif "Diag" in i:
        temp1 = temp1.filter(temp1["rank"] <= diag)
    temp1 = temp1.groupby("prov_npi").agg(collect_set("array_val").alias(i+"_new"))
    temp1.createOrReplaceTempView("temp1")
    lead_gen_fil = spark.sql("select a.*,"+" b."+i+"_new"+" from lead_gen_fil a join temp1 b on a.prov_npi = b.prov_npi")
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")

DOS_P12M_PatCnt
CPT_P12M_Cnt
CPT_P9M_Cnt
CPT_P6M_Cnt
CPT_P3M_Cnt
Diag_P12M_Cnt
Diag_P9M_Cnt
Diag_P6M_Cnt
Diag_P3M_Cnt


In [121]:
from pyspark.sql.functions import explode,col
col = ['DOS_P12M_PatCnt']

for i in col:
    print(i)
    temp = lead_gen_fil1.select("prov_npi",explode(i).alias("array_val"))
    reg = "((\d+)|(\d+\.\d+))(?=:)"
    temp = temp.withColumn("key",regexp_extract(temp["array_val"],reg,1))
    temp = temp.withColumn("key",temp["key"].cast("float"))
    window = Window.partitionBy(temp['prov_npi']).orderBy(temp["key"].desc())
    temp1 = temp.select('*', rank().over(window).alias('rank'))
    temp1.createOrReplaceTempView("temp1")
    if "DOS" in i:
        temp1 = temp1.filter(temp1["rank"]<=dos)
        temp1.createOrReplaceTempView("temp1")
temp1.show(truncate=False)

DOS_P12M_PatCnt
+----------+------------+-----+----+
|prov_npi  |array_val   |key  |rank|
+----------+------------+-----+----+
|1780788919|644:20191014|644.0|1   |
|1780788919|644:20191004|644.0|1   |
|1780788919|632:20191011|632.0|3   |
|1902175441|398:20200721|398.0|1   |
|1902175441|375:20200513|375.0|2   |
|1902175441|353:20200512|353.0|3   |
+----------+------------+-----+----+



In [74]:
new_col_list = [x for x in lead_gen_fil.columns if "new" in x]
print("new_col:\n",new_col_list)
for i in new_col_list:
    i_sub = i[0:len(i)-4]
    lead_gen_fil = lead_gen_fil.drop(i_sub).withColumnRenamed(i,i_sub)


new_col:
 ['DOS_P12M_PatCnt_new', 'CPT_P12M_Cnt_new', 'CPT_P9M_Cnt_new', 'CPT_P6M_Cnt_new', 'CPT_P3M_Cnt_new', 'Diag_P12M_Cnt_new', 'Diag_P9M_Cnt_new', 'Diag_P6M_Cnt_new', 'Diag_P3M_Cnt_new']


In [75]:
lead_gen_fil.createOrReplaceTempView("lead_gen_fil")

# Selecting required columns

In [79]:
lead_release = lead_release.withColumn("feat_array",split(lead_release["Features"],","))
selected_cols = lead_release.select("feat_array").collect()[0][0]
lead_gen_sub = lead_gen_fil.select(selected_cols)
selected_cols

['prov_npi',
 'prov_name',
 'entity_type',
 'DOS_P12M_PatCnt',
 'CPT_P9M_Cnt',
 'Diag_P6M_Cnt']

In [122]:
lead_gen_sub.show(truncate=False)

+----------+---------------------+-----------+--------------------------------------------------------------------------------+----------------------------------------+-------------------------------------------+
|prov_npi  |prov_name            |entity_type|DOS_P12M_PatCnt                                                                 |CPT_P9M_Cnt                             |Diag_P6M_Cnt                               |
+----------+---------------------+-----------+--------------------------------------------------------------------------------+----------------------------------------+-------------------------------------------+
|1912963968|RACHEL  BREGMAN      |1          |[103:20201118, 118:20201119, 132:20201204, 157:20210113, 204:20201231]          |[703 : 99211, 576 : 99213, 370 : 99212] |[1447:Z20828, 278:Z23, 128:Z0000]          |
|1366523060|BRANDON M RHODES     |1          |[10:20201020, 184:20201021, 34:20201022, 4:20201023, 13:20201103]               |[83 : 90471, 107 : 99

In [15]:
import time
toc =time.time()
import os
import re
import pyspark
import pandas as pd
from functools import reduce
from pyspark.sql import SparkSession
from pyspark.sql.functions import max as _max, min as _min,round
from pyspark.sql.functions import *
from pyspark.sql.functions import col, sqrt,sum,avg,count,explode
from operator import add
from pyspark.sql.types import IntegerType
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.sql.functions import trim
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql.functions import *

spark.stop()

print("Initiating Spark Cluster")
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '20g'), 
                                   ('spark.executor.instances', '12'),
                                   ('spark.driver.cores','1'), 
                                   ('spark.executor.cores', '4'), 
                                   ('spark.sql.broadcastTimeout', '36000'),
                                   ('spark.yarn.queue', 'root.araadh_q1.uhc_oa_pi_dev_sq1'),
                                   ('spark.driver.memory','5g')])
spark = SparkSession.builder.appName("Model-C").config(conf=conf).enableHiveSupport().getOrCreate().newSession()

##############################################################################################################################

#Lead Repository File
lead_gen = spark.read.parquet("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HNP_HSD/Final_HNP")
#Select the hnp leads 
lead_gen = lead_gen.filter(col("hnp_flag")=="yes")
lead_gen.createOrReplaceTempView("lead_gen")

#Lead Release File
lead_release = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HBP_Lead_Release_new.csv",header=True)

#Get the priority orders of teams-priority order determines to which team leads should be shared first
lead_release = lead_release.withColumn("Priority_Order",lead_release["Priority_Order"].cast(IntegerType()))
po = lead_release.select("Priority_Order")
priority_order = [int(row.Priority_Order) for row in po.collect()]
priority_order.sort()

#
for po in priority_order:
    lead_gen_new=lead_gen
    
    #Deduping the shared leads
    dedupe_file = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/de-dupe.csv",header=True)
    print("Dedupe_File")
    dedupe_file.show()
    lead_release_new = lead_release.filter(lead_release["priority_order"]==po)
    team = lead_release_new.select("Team").collect()[0][0]
    print("Team: ",team)
    dedupe = lead_release_new.select("Dedupe").collect()[0][0]
    print("Dedupe: ",dedupe)
    if(dedupe=="All"):
        lead_gen_new = lead_gen_new.join(dedupe_file,lead_gen_new.prov_npi==dedupe_file.prov_npi, how="leftanti")
    else:
        dedupe_file = dedupe_file.withColumn("flag",col("Team").rlike(i))
        dedupe_file_sub = dedupe_file.filter(col("flag")==True)
        lead_gen_new = lead_gen_new.join(dedupe_file_sub,lead_gen_new.prov_npi==dedupe_file_sub.prov_npi, how="leftanti")
    lead_gen_new.createOrReplaceTempView("lead_gen_new") 
    
    print("Categorical Filters")
    lead_release_new = lead_release_new.withColumn("catfil_arr",split(lead_release_new["Categorical_Filters"],";"))
    cat_filter = lead_release_new.select("catfil_arr").collect()[0][0]
    cat_filter

    col_name = ""
    col_value = ""
    if not (cat_filter is None):
        for i in cat_filter:
            col_name = i.split(":")[0].strip()
            col_value = i.split(":")[1].strip()
            col_value = col_value[1:len(col_value)-1]
            col_value_list = col_value.split(",")
            lead_gen_new = lead_gen_new.withColumn("cat_flag_"+col_name,concat_ws(",",lead_gen_new[col_name]))
            for j in col_value_list:
                print("j: ",j)
                lead_gen_new = lead_gen_new.withColumn("categorical_flag_"+j, lead_gen_new["cat_flag_"+col_name].rlike(j))
                lead_gen_new = lead_gen_new.withColumn("categorical_flag_"+j, lead_gen_new["categorical_flag_"+j].cast(StringType()))
    else:
        print("Category Filter is Empty")

    print("Numerical Filters")
    lead_release_new = lead_release_new.withColumn("num_fil_arr",split(lead_release_new["Numeric_Filters"],";"))
    num_filter = lead_release_new.select("num_fil_arr").collect()[0][0]
    num_filter

    
    col_name = ""
    col_value = ""
    if not(num_filter is None):
        for i in num_filter:
            i = i.strip()
            col_name = i.split(":")[0].strip()
            col_value = i.split(":")[1].strip()
            col_value = col_value[1:len(col_value)-1]
            col_value_list = col_value.split(",")
            col_str =col_name+"_str"
            print("col_name: ",col_name)
            print("col_value: ",col_value)
            print("col_value_list: ",col_value_list)
            lead_gen_new = lead_gen_new.withColumn(col_str,lead_gen_new[col_name].cast(StringType()))
            for j in col_value_list:
                j_list = re.split('([^a-zA-Z0-9])',j)
                col_name = j_list[0].strip()
                sign = j_list[1].strip()
                num = j_list[2].strip()
                #val = col_name[0:len(col_name)-1]
                #num = j.split(" ")[1].strip()
                #num = float(num)
                reg ="((\d+)|(\d+\.\d+))(?=:"+col_name+")" 
                #print("j_list :",j_list)
                #print("col_name: ",col_name)
                #print("cl:",cl)
                #print("val",val)
                #print("num",num)
                #print("sign",sign)
                #print("num",num)
                #a =cl+"_str"
                lead_gen_new = lead_gen_new.withColumn("num_"+col_name,regexp_extract(lead_gen_new[col_str],reg,1))
                lead_gen_new = lead_gen_new.withColumn("num_"+col_name,lead_gen_new["num_"+col_name].cast('float'))
                lead_gen_new = lead_gen_new.fillna(0)
                if sign==">":
                    lead_gen_new = lead_gen_new.withColumn("numeric_flag_"+col_name, when(lead_gen_new["num_"+col_name] >= num,"true").otherwise("false"))
                else:
                    lead_gen_new = lead_gen_new.withColumn("numeric_flag_"+col_name, when(lead_gen_new["num_"+col_name] <= num,"true").otherwise("false"))
    else:
        print("Numeric Filter is empty")
        

    lead_gen_new_columns = lead_gen_new.columns
    
    #Apply the filters to the lead repository file
    fil_col_list = [x for x in lead_gen_new_columns if ("categorical_flag" in x or "numeric_flag" in x)]
    print("flag_col_list:\n",fil_col_list)
    filter_query=""
    if (len(fil_col_list)!=0):
        for i in fil_col_list:
            lead_gen_new = lead_gen_new.withColumn(i,when(lead_gen_new[i]=="true","1").otherwise("0"))
            lead_gen_new.createOrReplaceTempView("lead_gen_new")
            filter_query =filter_query+" and "+i+" ==0"
        filter_query=filter_query[5:len(filter_query)]
        print("filter_query: ",filter_query)       
        lead_gen_fil = spark.sql("select a.* from lead_gen_new a where "+filter_query)
    else:
        lead_gen_fil = spark.sql("select * from lead_gen_new")
    print("#Leads after filtering",lead_gen_fil.count())
    #Extracting top CPT,DOS,DIAG etc. from the lead release file
    dos = int(lead_release_new.select("DOS").collect()[0][0])
    diag = int(lead_release_new.select("DIAG").collect()[0][0])
    cpt = int(lead_release_new.select("CPT").collect()[0][0])
    leads= int(lead_release_new.select("Leads").collect()[0][0])
    
    #Sorting the leads
    lead_gen_fil =lead_gen_fil.orderBy(lead_gen_fil["days_pat_grt_51"].desc(),lead_gen_fil["Avg_Max_PatCnt_Ratio"].desc()).limit(leads)
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
    print("#Leads extracted for",team,lead_gen_fil.count())
    
    array_columns = ['DOS_P12M_PatCnt','CPT_P12M_Cnt','CPT_P9M_Cnt','CPT_P6M_Cnt','CPT_P3M_Cnt','Diag_P12M_Cnt','Diag_P9M_Cnt','Diag_P6M_Cnt','Diag_P3M_Cnt']
    #Extracting top cpt,diag etc. from the array columns
    for i in array_columns:
        print(i)
        temp = lead_gen.select("prov_npi",explode(i).alias("array_val"))
        temp = temp.withColumn("key",trim(split(temp["array_val"],":").getItem(0)))
        temp = temp.withColumn("key",temp["key"].cast("float"))
        window = Window.partitionBy(temp['prov_npi']).orderBy(temp["key"].desc())
        temp1 = temp.select('*', rank().over(window).alias('rank'))
        if "DOS" in i:
            temp1 = temp1.filter(temp1["rank"] <= dos)
        elif "CPT" in i:
            temp1 = temp1.filter(temp1["rank"] <= cpt)
        elif "Diag" in i:
            temp1 = temp1.filter(temp1["rank"] <= diag)
        temp1 = temp1.groupby("prov_npi").agg(collect_set("array_val").alias(i+"_new"))
        temp1.createOrReplaceTempView("temp1")
        lead_gen_fil = spark.sql("select a.*,"+" b."+i+"_new"+" from lead_gen_fil a left join temp1 b on a.prov_npi = b.prov_npi")
        lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
    
    #Fetching the columns required by the team
    new_col = [x for x in lead_gen_fil.columns if "new" in x]
    print("new_col:\n",new_col)
    
    for i in new_col:
        i_sub = i[0:len(i)-4]
        lead_gen_fil = lead_gen_fil.drop(i_sub).withColumnRenamed(i,i_sub)
        lead_gen_fil = lead_gen_fil.withColumn(i_sub,f.col(i_sub).cast(StringType()))
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
       
    lead_release = lead_release.withColumn("feat_arr",split(lead_release["Features"],","))
    selected_cols = lead_release.select("feat_arr").collect()[0][0]
    lead_gen_sub = lead_gen_fil.select(selected_cols)
    #lead_gen_sub.cache()
    print("writing file")
    wrt_toc=time.time()
    lead_gen_sub.coalesce(1).write.mode("overwrite").option("header","true").csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HNP_HSD/"+str(team)+"hnp.csv")
    print("Done writing file")
    wrt_tic=time.time()
    #print("Time taken in writing the file is: ",(float(wrt_tic)-float(wrt_toc))/60)
    #lead_gen_sub.unpersist()
    #Removing the leads that are shared with one team
    lead_gen_sub.createOrReplaceTempView("lead_gen_sub")
    lead = lead_gen_sub.select("prov_npi")
    lead.createOrReplaceTempView("lead")
    print("Removing the shared leads from lead gen")
    lead_gen = lead_gen.join(lead,lead_gen.prov_npi==lead.prov_npi,how="leftanti")    
    lead_gen.createOrReplaceTempView("lead_gen")
    #lead_gen.select("prov_npi").show()
    print("######################","End of",po,"iteration","#######################")
tic =time.time()
tic=float(tic)
toc =float(toc)
print("Total time taken:",(tic-toc)/60)

Initiating Spark Cluster
Dedupe_File
+----------+----+
|  prov_npi|Team|
+----------+----+
|1649218173|PLRT|
|1427293281| AVS|
|1275791832|PLRT|
|1285660241| AVS|
+----------+----+

Team:  PLRT
Dedupe:  All
Categorical Filters
j:  PNIC
j:  95964
j:  MEDICA
Numerical Filters
Numeric Filter is empty
flag_col_list:
 ['categorical_flag_PNIC', 'categorical_flag_95964', 'categorical_flag_MEDICA']
filter_query:  categorical_flag_PNIC ==0 and categorical_flag_95964 ==0 and categorical_flag_MEDICA ==0
#Leads after filtering 166
#Leads extracted for PLRT 10
DOS_P12M_PatCnt
CPT_P12M_Cnt
CPT_P9M_Cnt
CPT_P6M_Cnt
CPT_P3M_Cnt
Diag_P12M_Cnt
Diag_P9M_Cnt
Diag_P6M_Cnt
Diag_P3M_Cnt
new_col:
 ['DOS_P12M_PatCnt_new', 'CPT_P12M_Cnt_new', 'CPT_P9M_Cnt_new', 'CPT_P6M_Cnt_new', 'CPT_P3M_Cnt_new', 'Diag_P12M_Cnt_new', 'Diag_P9M_Cnt_new', 'Diag_P6M_Cnt_new', 'Diag_P3M_Cnt_new']
writing file
Done writing file
Removing the shared leads from lead gen
###################### End of 1 iteration #######################

In [14]:
#Lead Repository File
lead_gen = spark.read.parquet("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HNP_HSD/Final_HNP")
#Select the hnp leads 
lead_gen = lead_gen.filter(col("hnp_flag")=="yes")
lead_gen.createOrReplaceTempView("lead_gen")
lead_gen.count()

#Lead Release File
lead_release = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HBP_Lead_Release_new.csv",header=True)

priority_order=[1]
for po in priority_order:
    lead_gen_new=lead_gen
    
    #Deduping the shared leads
    dedupe_file = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/de-dupe.csv",header=True)
    print("Dedupe_File")
    dedupe_file.show()
    lead_release_new = lead_release.filter(lead_release["priority_order"]==po)
    team = lead_release_new.select("Team").collect()[0][0]
    print("Team: ",team)
    dedupe = lead_release_new.select("Dedupe").collect()[0][0]
    print("Dedupe: ",dedupe)
    if(dedupe=="All"):
        lead_gen_new = lead_gen_new.join(dedupe_file,lead_gen_new.prov_npi==dedupe_file.prov_npi, how="leftanti")
    else:
        dedupe_file = dedupe_file.withColumn("flag",col("Team").rlike(i))
        dedupe_file_sub = dedupe_file.filter(col("flag")==True)
        lead_gen_new = lead_gen_new.join(dedupe_file_sub,lead_gen_new.prov_npi==dedupe_file_sub.prov_npi, how="leftanti")
    lead_gen_new.createOrReplaceTempView("lead_gen_new") 
    
    print("Categorical Filters")
    lead_release_new = lead_release_new.withColumn("catfil_arr",split(lead_release_new["Categorical_Filters"],";"))
    cat_filter = lead_release_new.select("catfil_arr").collect()[0][0]
    cat_filter

    col_name = ""
    col_value = ""
    if not (cat_filter is None):
        for i in cat_filter:
            col_name = i.split(":")[0].strip()
            col_value = i.split(":")[1].strip()
            col_value = col_value[1:len(col_value)-1]
            col_value_list = col_value.split(",")
            lead_gen_new = lead_gen_new.withColumn("cat_flag_"+col_name,concat_ws(",",lead_gen_new[col_name]))
            for j in col_value_list:
                print("j: ",j)
                lead_gen_new = lead_gen_new.withColumn("categorical_flag_"+j, lead_gen_new["cat_flag_"+col_name].rlike(j))
                lead_gen_new = lead_gen_new.withColumn("categorical_flag_"+j, lead_gen_new["categorical_flag_"+j].cast(StringType()))
    else:
        print("Category Filter is Empty")

    print("Numerical Filters")
    lead_release_new = lead_release_new.withColumn("num_fil_arr",split(lead_release_new["Numeric_Filters"],";"))
    num_filter = lead_release_new.select("num_fil_arr").collect()[0][0]
    num_filter

    
    col_name = ""
    col_value = ""
    if not(num_filter is None):
        for i in num_filter:
            i = i.strip()
            col_name = i.split(":")[0].strip()
            col_value = i.split(":")[1].strip()
            col_value = col_value[1:len(col_value)-1]
            col_value_list = col_value.split(",")
            col_str =col_name+"_str"
            print("col_name: ",col_name)
            print("col_value: ",col_value)
            print("col_value_list: ",col_value_list)
            lead_gen_new = lead_gen_new.withColumn(col_str,lead_gen_new[col_name].cast(StringType()))
            for j in col_value_list:
                j_list = re.split('([^a-zA-Z0-9])',j)
                col_name = j_list[0].strip()
                sign = j_list[1].strip()
                num = j_list[2].strip()
                #val = col_name[0:len(col_name)-1]
                #num = j.split(" ")[1].strip()
                #num = float(num)
                reg ="((\d+)|(\d+\.\d+))(?=:"+col_name+")" 
                #print("j_list :",j_list)
                #print("col_name: ",col_name)
                #print("cl:",cl)
                #print("val",val)
                #print("num",num)
                #print("sign",sign)
                #print("num",num)
                #a =cl+"_str"
                lead_gen_new = lead_gen_new.withColumn("num_"+col_name,regexp_extract(lead_gen_new[col_str],reg,1))
                lead_gen_new = lead_gen_new.withColumn("num_"+col_name,lead_gen_new["num_"+col_name].cast('float'))
                lead_gen_new = lead_gen_new.fillna(0)
                if sign==">":
                    lead_gen_new = lead_gen_new.withColumn("numeric_flag_"+col_name, when(lead_gen_new["num_"+col_name] >= num,"true").otherwise("false"))
                else:
                    lead_gen_new = lead_gen_new.withColumn("numeric_flag_"+col_name, when(lead_gen_new["num_"+col_name] <= num,"true").otherwise("false"))
    else:
        print("Numeric Filter is empty")
        

    lead_gen_new_columns = lead_gen_new.columns
    
    #Apply the filters to the lead repository file
    fil_col_list = [x for x in lead_gen_new_columns if ("categorical_flag" in x or "numeric_flag" in x)]
    print("flag_col_list:\n",fil_col_list)
    filter_query=""
    if (len(fil_col_list)!=0):
        for i in fil_col_list:
            lead_gen_new = lead_gen_new.withColumn(i,when(lead_gen_new[i]=="true","1").otherwise("0"))
            lead_gen_new.createOrReplaceTempView("lead_gen_new")
            filter_query =filter_query+" and "+i+" ==0"
        filter_query=filter_query[5:len(filter_query)]
        print("filter_query: ",filter_query)       
        lead_gen_fil = spark.sql("select a.* from lead_gen_new a where "+filter_query)
    else:
        lead_gen_fil = spark.sql("select * from lead_gen_new")
    print("#Leads after filtering",lead_gen_fil.count())
    #Extracting top CPT,DOS,DIAG etc. from the lead release file
    dos = int(lead_release_new.select("DOS").collect()[0][0])
    diag = int(lead_release_new.select("DIAG").collect()[0][0])
    cpt = int(lead_release_new.select("CPT").collect()[0][0])
    leads= int(lead_release_new.select("Leads").collect()[0][0])
    
    #Sorting the leads
    lead_gen_fil =lead_gen_fil.orderBy(lead_gen_fil["days_pat_grt_51"].desc(),lead_gen_fil["Avg_Max_PatCnt_Ratio"].desc()).limit(leads)
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
    print("#Leads extracted for",team,lead_gen_fil.count())
    print("1")
    lead_gen_fil.select("prov_npi").show()
    
    array_columns = ['DOS_P12M_PatCnt','CPT_P12M_Cnt','CPT_P9M_Cnt','CPT_P6M_Cnt','CPT_P3M_Cnt','Diag_P12M_Cnt','Diag_P9M_Cnt','Diag_P6M_Cnt','Diag_P3M_Cnt']
    #Extracting top cpt,diag etc. from the array columns
    for i in array_columns:
        print(i)
        temp = lead_gen.select("prov_npi",explode(i).alias("array_val"))
        temp = temp.withColumn("key",trim(split(temp["array_val"],":").getItem(0)))
        temp = temp.withColumn("key",temp["key"].cast("float"))
        window = Window.partitionBy(temp['prov_npi']).orderBy(temp["key"].desc())
        temp1 = temp.select('*', rank().over(window).alias('rank'))
        if "DOS" in i:
            temp1 = temp1.filter(temp1["rank"] <= dos)
        elif "CPT" in i:
            temp1 = temp1.filter(temp1["rank"] <= cpt)
        elif "Diag" in i:
            temp1 = temp1.filter(temp1["rank"] <= diag)
        temp1 = temp1.groupby("prov_npi").agg(collect_set("array_val").alias(i+"_new"))
        temp1.createOrReplaceTempView("temp1")
        lead_gen_fil = spark.sql("select a.*,"+" b."+i+"_new"+" from lead_gen_fil a left join temp1 b on a.prov_npi = b.prov_npi")
        lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
    print("2")
    lead_gen_fil.select("prov_npi").show(truncate=False)

    #Fetching the columns required by the team
    new_col = [x for x in lead_gen_fil.columns if "new" in x]
    print("new_col:\n",new_col)
    
    for i in new_col:
        i_sub = i[0:len(i)-4]
        lead_gen_fil = lead_gen_fil.drop(i_sub).withColumnRenamed(i,i_sub)
        lead_gen_fil = lead_gen_fil.withColumn(i_sub,f.col(i_sub).cast(StringType()))
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
       
    lead_release = lead_release.withColumn("feat_arr",split(lead_release["Features"],","))
    selected_cols = lead_release.select("feat_arr").collect()[0][0]
    lead_gen_sub = lead_gen_fil.select(selected_cols)
    print("3")
    lead_gen_sub.select("prov_npi").show()
    
    lead_gen_sub.coalesce(1).write.mode("overwrite").option("header","true").csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HNP_HSD/"+str(team)+"hnp.csv")

    
    

Dedupe_File
+----------+----+
|  prov_npi|Team|
+----------+----+
|1649218173|PLRT|
|1427293281| AVS|
|1275791832|PLRT|
|1285660241| AVS|
+----------+----+

Team:  PLRT
Dedupe:  All
Categorical Filters
j:  PNIC
j:  95964
j:  MEDICA
Numerical Filters
Numeric Filter is empty
flag_col_list:
 ['categorical_flag_PNIC', 'categorical_flag_95964', 'categorical_flag_MEDICA']
filter_query:  categorical_flag_PNIC ==0 and categorical_flag_95964 ==0 and categorical_flag_MEDICA ==0
#Leads after filtering 166
#Leads extracted for PLRT 10
1
+----------+
|  prov_npi|
+----------+
|1801971502|
|1053609156|
|1154589539|
|1679803340|
|1255379772|
|1255388674|
|1528257649|
|1174795314|
|1518126655|
|1033698246|
+----------+

DOS_P12M_PatCnt
CPT_P12M_Cnt
CPT_P9M_Cnt
CPT_P6M_Cnt
CPT_P3M_Cnt
Diag_P12M_Cnt
Diag_P9M_Cnt
Diag_P6M_Cnt
Diag_P3M_Cnt
2
+----------+
|prov_npi  |
+----------+
|1801971502|
|1053609156|
|1154589539|
|1679803340|
|1255379772|
|1255388674|
|1528257649|
|1174795314|
|1518126655|
|103369824

In [ ]:
#col = ['CPT_P12M_Cnt','CPT_P9M_Cnt','CPT_P6M_Cnt','CPT_P3M_Cnt','Diag_P12M_Cnt','Diag_P9M_Cnt','Diag_P6M_Cnt','Diag_P3M_Cnt','DOS_P12M_PatCnt']
    
    from pyspark.sql.functions import explode,col
    array_columns = ['DOS_P12M_PatCnt','CPT_P12M_Cnt','CPT_P9M_Cnt','CPT_P6M_Cnt','CPT_P3M_Cnt','Diag_P12M_Cnt','Diag_P9M_Cnt','Diag_P6M_Cnt','Diag_P3M_Cnt']

    for i in array_columns:
        #print(i)
        temp = lead_gen.select("prov_npi",explode(i).alias("array_val"))
        temp = temp.withColumn("key",trim(split(temp["array_val"],":").getItem(0)))
        temp = temp.withColumn("key",temp["key"].cast("float"))
        window = Window.partitionBy(temp['prov_npi']).orderBy(temp["key"].desc())
        temp1 = temp.select('*', rank().over(window).alias('rank'))
        if "DOS" in i:
            temp1 = temp1.filter(temp1["rank"] <= dos)
        elif "CPT" in i:
            temp1 = temp1.filter(temp1["rank"] <= cpt)
        elif "Diag" in i:
            temp1 = temp1.filter(temp1["rank"] <= diag)
        temp1 = temp1.groupby("prov_npi").agg(collect_set("array_val").alias(i+"_new"))
        temp1.createOrReplaceTempView("temp1")
        lead_gen_fil = spark.sql("select a.*,"+" b."+i+"_new"+" from lead_gen_fil a join temp1 b on a.prov_npi = b.prov_npi")
        lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
        
    new_col = [x for x in lead_gen_fil.columns if "new" in x]
    print("new_col:\n",new_col)
    
    for i in new_col:
        i_sub = i[0:len(i)-4]
        lead_gen_fil = lead_gen_fil.drop(i_sub).withColumnRenamed(i,i_sub)
        lead_gen_fil = lead_gen_fil.withColumn(i_sub,f.col(i_sub).cast(StringType()))
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
    #lead_gen_fil.printSchema()
       
    lead_release = lead_release.withColumn("feat_arr",split(lead_release["Features"],","))
    selected_cols = lead_release.select("feat_arr").collect()[0][0]
    lead_gen_sub = lead_gen_fil.select(selected_cols)
    lead_gen_sub.cache()
    print("writing file")
    wrt_toc=time.time()
    lead_gen_sub.coalesce(1).write.mode("overwrite").csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/"+str(team)+".csv")
    print("Done writing file")
    wrt_tic=time.time()
    print("Time taken in writing the file is: ",(float(wrt_tic)-float(wrt_toc))/60)
    lead_gen_sub.unpersist()
    #Removing the leads that are shared with one team
    lead_gen_sub.createOrReplaceTempView("lead_gen_sub")
    lead = lead_gen_sub.select("prov_npi")
    lead.createOrReplaceTempView("lead")
    print("Removing the shared leads from lead gen")
    lead_gen = lead_gen.join(lead,lead_gen.prov_npi==lead.prov_npi,how="leftanti")    
    lead_gen.createOrReplaceTempView("lead_gen")
    #lead_gen.select("prov_npi").show()
    print("######################","End of",po,"iteration","#######################")

# Dry Run

In [187]:
lead_gen.columns

['prov_npi',
 'entity_type',
 'DOS_P12M_PatCnt',
 'Reference_Date',
 'Platform_P12M_ClmPer',
 'CPT_P12M_Cnt',
 'CPT_P9M_Cnt',
 'CPT_P6M_Cnt',
 'CPT_P3M_Cnt',
 'Avg_P3M_PatCnt',
 'Avg_P6M_PatCnt',
 'Avg_P9M_PatCnt',
 'Avg_P12M_PatCnt',
 'Diag_P12M_Cnt',
 'Diag_P9M_Cnt',
 'Diag_P6M_Cnt',
 'Diag_P3M_Cnt',
 'Taxonomy_P12M_ClmPer',
 'Payer_ID_P12M_ClmPer',
 'Payer_nm_P12M_ClmPer',
 'Avg_Max_PatCnt_Ratio',
 'json_col',
 'cat_flag_Platform_P12M_ClmPer',
 'fil_cat_PNIC',
 'fil_cat_CIRR',
 'cat_flag_Taxonomy_P12M_ClmPer',
 'fil_cat_208100000X',
 'Platform_P12M_ClmPer_str',
 'num_PNIC',
 'fil_num_PNIC',
 'num_CIRR',
 'fil_num_CIRR']

In [1]:
time = time.time()
import os
import pyspark
import pandas as pd
from functools import reduce
from pyspark.sql import SparkSession
from pyspark.sql.functions import max as _max, min as _min,round
from pyspark.sql.functions import *
from pyspark.sql.functions import col, sqrt,sum,avg,count,explode
from operator import add
from pyspark.sql.types import IntegerType
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.sql.functions import trim
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql.functions import *



conf = pyspark.SparkConf().setAll([('spark.executor.memory', '20g'), 
                                   ('spark.executor.instances', '12'),
                                   ('spark.driver.cores','1'), 
                                   ('spark.executor.cores', '4'), 
                                   ('spark.sql.broadcastTimeout', '36000'),
                                   ('spark.yarn.queue', 'root.araadh_q1.uhc_oa_pi_dev_sq1'),
                                   ('spark.driver.memory','5g')])
spark = SparkSession.builder.appName("Model-C").config(conf=conf).enableHiveSupport().getOrCreate().newSession()

lead_gen = spark.read.parquet("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HBP_Lead_Gen")
dedupe = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/de-dupe.csv",header=True)
lead_gen = lead_gen.join(dedupe,lead_gen.prov_npi==dedupe.prov_npi, how="left_anti")
lead_gen.createOrReplaceTempView("lead_gen")

lr = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HBP_Lead_Release.csv",header=True)
lr = lr.withColumn("cf_arr",split(lr["Categorical_Filters"],","))
a_cat = lr.select("cf_arr").collect()[0][0]

cl = ""
vl = ""
if not (a_cat is None):
    for i in a_cat:
        cl = i.split(" ")[0].strip()
        vl = i.split(" ")[1].strip()
        print(i)
        print(cl)
        print(vl)
        lead_gen = lead_gen.withColumn("cat_flag_"+cl,concat_ws(",",lead_gen[cl]))
        lead_gen = lead_gen.withColumn("fil_cat_"+vl, lead_gen["cat_flag_"+cl].rlike(vl))
else:
    print("Category Filter is Empty")

lr = lr.withColumn("cf_arr",split(lr["Numeric_Filters"],","))
a_num = lr.select("cf_arr").collect()[0][0]
a_num

cl = ""
vl = ""
if not(a_num is None):
    for i in a_num:
        i = i.strip()
        cl = i.split(" ")[0].strip()
        vl = i.split(" ")[1].strip()
        sign = i.split(" ")[2].strip()
        num = float(i.split(" ")[3].strip())
        reg ="((\d+)|(\d+\.\d+))(?=:"+vl+")" 
        print(i.split())
        print(i)
        print(cl)
        print(vl)
        print(num)
        print(sign)
        a =cl+"_str"
        lead_gen = lead_gen.withColumn(a,lead_gen[cl].cast(StringType()))
        lead_gen = lead_gen.withColumn("num_"+vl,regexp_extract(lead_gen[a],reg,1))
        lead_gen = lead_gen.withColumn("num_"+vl,lead_gen["num_"+vl].cast('float'))
        if sign==">":
            lead_gen = lead_gen.withColumn("fil_num_"+vl, when(lead_gen["num_"+vl] >= num,"true").otherwise("false"))
        else:
            lead_gen = lead_gen.withColumn("fil_num_"+vl, when(lead_gen["num_"+vl] <= num,"true").otherwise("false"))
else:
    print("Numeric Filter is empty")

lead_gen.createOrReplaceTempView("lead_gen")

lead_gen_columns = lead_gen.columns

fil_col = [x for x in lead_gen_columns if "fil" in x]
print("fil_col:\n",fil_col)
for i in fil_col:
    lead_gen = lead_gen.withColumn(i,when(lead_gen[i]=="true","exclude").otherwise("include"))

lead_gen.createOrReplaceTempView("lead_gen")

q=""
for i in fil_col:
    q =q+" and "+i+" =='include'"
qf=q[5:len(q)]
print(qf)

lead_gen_fil = spark.sql("select a.* from lead_gen a where "+qf)

dos = int(lr.select("DOS").collect()[0][0])
diag = int(lr.select("DIAG").collect()[0][0])
cpt = int(lr.select("CPT").collect()[0][0])
leads= int(lr.select("Leads").collect()[0][0])

lead_gen_fil =lead_gen_fil.orderBy(lead_gen_fil["Avg_Max_PatCnt_Ratio"].desc()).limit(leads)
lead_gen_fil.createOrReplaceTempView("lead_gen_fil")

from pyspark.sql.functions import explode,col
col = ['DOS_P12M_PatCnt','CPT_P12M_Cnt','CPT_P9M_Cnt','CPT_P6M_Cnt','CPT_P3M_Cnt','Diag_P12M_Cnt','Diag_P9M_Cnt','Diag_P6M_Cnt','Diag_P3M_Cnt']

for i in col:
    print(i)
    temp = lead_gen.select("prov_npi",explode(i).alias("array_val"))
    temp = temp.withColumn("key",trim(split(temp["array_val"],":").getItem(0)))
    temp = temp.withColumn("key",temp["key"].cast("float"))
    window = Window.partitionBy(temp['prov_npi']).orderBy(temp["key"].desc())
    temp1 = temp.select('*', rank().over(window).alias('rank'))
    if "DOS" in i:
        temp1 = temp1.filter(temp1["rank"] <= dos)
    elif "CPT" in i:
        temp1 = temp1.filter(temp1["rank"] <= cpt)
    elif "Diag" in i:
        temp1 = temp1.filter(temp1["rank"] <= diag)
    temp1 = temp1.groupby("prov_npi").agg(collect_set("array_val").alias(i+"_new"))
    temp1.createOrReplaceTempView("temp1")
    lead_gen_fil = spark.sql("select a.*,"+" b."+i+"_new"+" from lead_gen_fil a join temp1 b on a.prov_npi = b.prov_npi")
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")

new_col = [x for x in lead_gen_fil.columns if "new" in x]
print("new_col:\n",new_col)
for i in new_col:
    i_sub = i[0:len(i)-4]
    lead_gen_fil = lead_gen_fil.drop(i_sub).withColumnRenamed(i,i_sub)

lead_gen_fil.createOrReplaceTempView("lead_gen_fil")

lr = lr.withColumn("feat_arr",split(lr["Features"],","))
selected_cols = lr.select("feat_arr").collect()[0][0]
lead_gen_sub = lead_gen_fil.select(selected_cols)

btime=time.time()
btime-atime

NameError: name 'time' is not defined

In [22]:
from pyspark.sql.functions import explode,col
col = ['DOS_PatCnt_P12M','CPT_P12M','CPT_P9M','CPT_P6M','CPT_P3M','Diag_Cnt_P12M','Diag_Cnt_P9M','Diag_Cnt_P6M','Diag_Cnt_P3M']

for i in col:
    print(i)
    temp = lead_gen.select("prov_npi",explode(i).alias("array_val"))
    reg = "((\d+)|(\d+\.\d+))(?=:)"
    temp = temp.withColumn("key",regexp_extract(temp["array_val"],reg,1))
    temp = temp.withColumn("key",temp["key"].cast("float"))
    window = Window.partitionBy(temp['prov_npi']).orderBy(temp["key"].desc())
    temp1 = temp.select('*', rank().over(window).alias('rank'))
    temp1 = temp1.filter(temp1["rank"] <= dos)
    temp1 = temp1.groupby("prov_npi").agg(collect_set("array_val").alias(i+"_new"))
    temp1.createOrReplaceTempView("temp1")
    lead_gen = spark.sql("select a.*,"+" b."+i+"_new"+" from lead_gen a join temp1 b on a.prov_npi = b.prov_npi")
    lead_gen.createOrReplaceTempView("lead_gen_fil")

DOS_PatCnt_P12M
CPT_P12M
CPT_P9M
CPT_P6M
CPT_P3M
Diag_Cnt_P12M
Diag_Cnt_P9M
Diag_Cnt_P6M
Diag_Cnt_P3M


In [ ]:
cl = ""
vl = ""
if not (a_cat is None):
    for i in a_cat:
        cl = i.split(" ")[0].strip()
        vl = i.split(" ")[1].strip()
        print(i)
        print(cl)
        print(vl)
        lead_gen = lead_gen.withColumn("cat_flag_"+cl,concat_ws(",",lead_gen[cl]))
        lead_gen = lead_gen.withColumn("fil_cat_"+vl, lead_gen["cat_flag_"+cl].rlike(vl))
else:
    print("Category Filter is Empty")

In [ ]:
cl = ""
vl = ""
if not(a_num is None):
    for i in a_num:
        i = i.strip()
        cl = i.split(" ")[0].strip()
        vl = i.split(" ")[1].strip()
        sign = i.split(" ")[2].strip()
        num = float(i.split(" ")[3].strip())
        reg ="((\d+)|(\d+\.\d+))(?=:"+vl+")" 
        print(i.split())
        print(i)
        print(cl)
        print(vl)
        print(num)
        print(sign)
        a =cl+"_str"
        #lead_gen = lead_gen.withColumn(a,lead_gen[cl].cast(StringType()))
        #lead_gen = lead_gen.withColumn("num_"+vl,regexp_extract(lead_gen[a],reg,1))
        #lead_gen = lead_gen.withColumn("num_"+vl,lead_gen["num_"+vl].cast('float'))
        if sign==">":
            #lead_gen = lead_gen.withColumn("fil_num_"+vl, when(lead_gen["num_"+vl] >= num,"true").otherwise("false"))
        else:
            #lead_gen = lead_gen.withColumn("fil_num_"+vl, when(lead_gen["num_"+vl] <= num,"true").otherwise("false"))
else:
    print("Numeric Filter is empty")

In [ ]:
#print("fil_col:\n",fil_col)
    #print("type",type(fil_col))
    if (len(fil_col)!=0):
        #print("here")
        for i in fil_col:
            lead_gen_new = lead_gen_new.withColumn(i,when(lead_gen_new[i]=="true","exclude").otherwise("include"))
        lead_gen_new.createOrReplaceTempView("lead_gen_new")
        q=""
        for i in fil_col:
            q =q+" and "+i+" =='include'"
        qf=q[5:len(q)]
        #print(qf)
        lead_gen_fil = spark.sql("select a.* from lead_gen_new a where "+qf)
    else:
        lead_gen_fil = spark.sql("select * from lead_gen_new")